# 手写字符识别
   主要实现两个方面内容，用softmax做一个模型，再用2层的神经网络实现一个模型。  
   ** 运行环境为win10 + Cuda8.0 + CuDNN_v5.1 **  
   注意CuDNN_v5.1是必须的

首先从TensorFlow的例子中加载MNIST数据,图像的尺寸为28*28的灰度图  <br/>
这里的数据其实是来自于tensorflow.contrib.learn.python.learn.datasets.mnist中，这个可以看github中的代码知道   <br/>

** 简单介绍一下数据集：有训练集合测试集两部分,内容如下：**

| 文件         |内容       |
|:---------------:|:---------:|
|train-images-idx3-ubyte.gz  |训练集图片 - 55000张训练图片, 5000张验证图片  |
|train-labels-idx1-ubyte.gz  |训练集图片对应的数字标签 |
|t10k-images-idx3-ubyte.gz  |测试集图片 - 10000 张 图片 |
|t10k-labels-idx1-ubyte.gz  |测试集图片对应的数字标签  |


## 一、Softmax进行字符识别
分为如下几个部分：
1. 读取数据
2. 建立softmax回归模型用几千的样本训练
3. 准确度检验

由于softmax不考虑像素之间相关性，所以输入的数据直接变成一维的向量

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


上面读取过数据，接下来开始画图

In [21]:
#1、开始建立一个图
sess = tf.InteractiveSession()#启动一个交互会话
x = tf.placeholder(tf.float32, shape=[None, 784])#x和y_都用一个占位符表示
y_ = tf.placeholder(tf.float32, shape=[None, 10])

W = tf.Variable(tf.zeros([784, 10]))#W和b因为需要改变，所以定义为初始化为0的变量
b = tf.Variable(tf.zeros(10))

#2、建立预测部分的操作节点
y = tf.matmul(x,W) + b  #计算wx+b
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)) #计算softmax交叉熵的均值

#3、现在已经得到了损失函数，接下来要做的就是最小化这一损失函数，这里用最常用的梯度下降做
# 为了用到前几节说过的内容，这里用学习率随训练下降的方法执行
global_step = tf.Variable(0, trainable = False)#建立一个可变数，而且这个变量在计算梯度时候不被影响,其实就是个全局变量
start_learning_rate = 0.5#这么写是为了清楚
#得到所需的学习率，学习率每100个step进行一次变化，公式为decayed_learning_rate = learning_rate * decay_rate ^(global_step / decay_steps)
learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 10, 0.9, staircase=True)

train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)#梯度下降最小化交叉熵

上面把所有的建图过程都弄完了，接下来就是训练过程

In [22]:
#这是因为在交互的Session下可以这样写Op.run()，还可以sess.run(tf.global_variables_initializer())
tf.global_variables_initializer().run()#初始化所有变量

#iteration = 1000, Batch_Size = 100 
for _ in range(1000):
    batch = mnist.train.next_batch(100)#每次选出100个数据
    train_step.run(feed_dict = {x:batch[0], y_: batch[1]})#给Placeholder填充数据就可以了

训练完毕，现在需要做的是评估训练结果，这个其实是个很复杂的事，因为需要调整的地方很多，这里忽略大部分如交叉验证调参等问题
可以得到一个准确率约为0.92的结果，但是这个不是很理想

In [25]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1)) #首先比较两个结果的差异
#这时的correct_prediction应该类似[True, False, True, True]，然后只要转为float的形式再求加和平均就知道准确率了
#这里的cast是用于形式转化
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))
#打印出来就可以了，注意这个时候accuracy也只是一个tensor,而且也只是一个模型的代表，还需要输入数据
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

sess.close()

0.9189


## 二、2层的cnn神经网络
 因为上面的结果不是很理想，这里进一步的用卷积神经网络试一试
 这里首先要设计一下神经网络的构造：  
 
<font color=#DAA520  size=5>输入-->cov1-->relu-->max_pool-->cov2-->relu-->max_pool-->full_connect-->drop_out-->softmax-->full_connect-->结果</font>  

<table><tr><td bgcolor=orange>会用到许多tf.nn的内容</td></tr></table>  

步骤还是和上面一样定义变量
不过这个时候注意：
    * 尺寸变了，直接输入的是一个图像了
    * 不是用0来初始化变量W和b了，而是用截断高斯来定义
    * 初始化方法变了不是同一进行初始化，这是为了体现另一种初始化方法，实际也可以和上面一样

In [3]:
#首先把要重复用的定义好
def weight_variable(shape):
    initial = tf.truncated_normal(shape=shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)#常量转变量,
    return tf.Variable(initial)
def conv2d(x, f):
    return tf.nn.conv2d(x, f, strides=[1,1,1,1], padding='SAME')
def max_pool_22(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

首先构造图
这里的输入和上面一样，但是注意输入数据要恢复成一张图

In [4]:
sess = tf.InteractiveSession()#启动一个交互会话
x = tf.placeholder(tf.float32, shape=[None, 784])#x和y_都用一个占位符表示
y_ = tf.placeholder(tf.float32, shape=[None, 10])
x_image = tf.reshape(x, [-1, 28, 28, 1])
#第一层：
#1、设计卷积核1
fW1 = weight_variable([5,5,1,32])#[height, weight, in_channel, out_channel]
fb1 = bias_variable([32])

#2、卷积加池化
h1 = tf.nn.relu(conv2d(x_image,fW1)+ fb1)
h1_pool = max_pool_22(h1)

#第二层
fW2 = weight_variable([5,5,32,64])#[height, weight, in_channel, out_channel]
fb2 = bias_variable([64])

h2 = tf.nn.relu(conv2d(h1_pool,fW2)+ fb2)
h2_pool = max_pool_22(h2)

#全部变成一维全连接层，这里因为是按照官方走的，所以手动计算了经过第二层后的图片尺寸为7*7
#来定义了一个wx+b所需的w和b的尺寸，注意这里的W和b不是卷积所用的了
h2_pool_flat = tf.reshape(h2_pool, [-1, 7*7*64])#首先把数据变成行表示
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_fc1 = tf.nn.relu(tf.matmul(h2_pool_flat, W_fc1) + b_fc1)

#定义dropout,选择性失活,首先指定一个失活的比例
prob = tf.placeholder(tf.float32)
h_dropout = tf.nn.dropout(h_fc1, prob)

#最后一个全连接层，输出10个值，用于softmax
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_dropout, W_fc2) + b_fc2

#梯度更新，这里采用另一种优化方式AdamOptimizer
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


接下来正式的训练模型  
+ 这里记住dropout有prob的超参数,一般都是用0.5
+ 每100次迭代显示一次训练准确度的结果
+ 这里总共用了2000次迭代，每次用50个数据

In [14]:
#初始化
sess.run(tf.global_variables_initializer())
for i in range(2000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict = {x:batch[0],y_:batch[1], prob:1.0}) #这里是计算accuracy用的eval,不是在run一个Operation
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], prob: 0.5})
print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, prob: 1.0}) )

step 0, training accuracy 0.08
step 100, training accuracy 0.76
step 200, training accuracy 0.9
step 300, training accuracy 0.9
step 400, training accuracy 0.96
step 500, training accuracy 0.92
step 600, training accuracy 0.96
step 700, training accuracy 0.94
step 800, training accuracy 0.96
step 900, training accuracy 0.94
step 1000, training accuracy 0.96
step 1100, training accuracy 1
step 1200, training accuracy 0.94
step 1300, training accuracy 0.98
step 1400, training accuracy 0.96
step 1500, training accuracy 0.98
step 1600, training accuracy 0.94
step 1700, training accuracy 0.94
step 1800, training accuracy 0.98
step 1900, training accuracy 1
test accuracy 0.9796


这里的测试结果是test accuracy = 0.9796， 和官方的差一点点，因为我为了方便只用了2000次循环，官方教程用了20000次
而且我的GPU比较老是GTX745，但是训练速度还是很快的